In [3]:
import numpy as np
import pandas as pd
import json
import gc
import re
import os

# -------------------------------------------------------------------
# Kaggle-specific file paths
# Replace <your-dataset-folder> with the folder name in /kaggle/input
# -------------------------------------------------------------------
DATA_PATH = "/kaggle/input/intelligent-bank/"

# Load CSV files
transaction_df = pd.read_csv(os.path.join(DATA_PATH, "transactions_data_south_africa.csv"))
card_df = pd.read_csv(os.path.join(DATA_PATH, "cards_data_south_africa.csv"))
users_df = pd.read_csv(os.path.join(DATA_PATH, "user_data_south_africa.csv"))

# Load MCC JSON file
mcc_series = pd.read_json(os.path.join(DATA_PATH, "mcc_codes.json"), typ='series')
mcc_df = mcc_series.reset_index()
mcc_df.columns = ['mcc_code', 'description']

# Load labels JSON
file_path = os.path.join(DATA_PATH, 'train_fraud_labels.json')

with open(file_path, 'r') as f:
    raw_json_data = json.load(f)

transaction_labels_dict = raw_json_data['target']

train_fraud_labels = pd.Series(transaction_labels_dict).reset_index()
train_fraud_labels.columns = ['transaction_id', 'is_fraud']
train_fraud_labels['transaction_id'] = pd.to_numeric(train_fraud_labels['transaction_id'])

print("\nAll data files loaded successfully in Kaggle environment.")



All data files loaded successfully in Kaggle environment.


## Transaction csv data

In [2]:
transaction_df.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2022-02-19 05:51:55,1556,2972,-1386.00,Swipe Transaction,59935,Kimberley,Northern Cape,8300.0,5499.0,NaN
1,7475328,2023-01-13 02:58:58,561,4575,262.26,Swipe Transaction,67570,Pietermaritzburg,KwaZulu-Natal,3200.0,5311.0,NaN
2,7475329,2024-07-03 23:41:24,1129,102,1440.00,Swipe Transaction,27092,Port Elizabeth,Eastern Cape,6000.0,4829.0,NaN
3,7475331,2022-06-12 09:33:41,430,2860,3600.00,Swipe Transaction,27092,Bloemfontein,Free State,9300.0,4829.0,NaN
4,7475332,2023-08-26 10:05:48,848,3915,835.38,Swipe Transaction,13051,Polokwane,Limpopo,700.0,5813.0,NaN


In [6]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12858754 entries, 0 to 12858753
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          float64
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             float64
 11  errors          object 
dtypes: float64(3), int64(4), object(5)
memory usage: 1.1+ GB


In [5]:
transaction_df.describe()

,id,client_id,card_id,amount,merchant_id,zip,mcc
count,1.285875e+07,1.285875e+07,1.285875e+07,1.285875e+07,1.285875e+07,1.134923e+07,1.285875e+07
mean,1.530925e+07,1.026802e+03,3.475013e+03,7.739152e+02,4.772799e+04,4.017428e+03,5.565479e+03
std,4.544713e+06,5.816489e+02,1.674463e+03,1.471071e+03,2.581482e+04,3.176196e+03,8.755668e+02
min,7.475327e+06,0.000000e+00,0.000000e+00,-9.000000e+03,1.000000e+00,1.000000e+00,1.711000e+03
25%,1.136989e+07,5.190000e+02,2.413000e+03,1.607400e+02,2.588700e+04,1.200000e+03,5.300000e+03
50%,1.529639e+07,1.070000e+03,3.583000e+03,5.218200e+02,4.592800e+04,4.000000e+03,5.499000e+03
75%,1.924064e+07,1.530000e+03,4.905000e+03,1.147140e+03,6.757000e+04,8.000000e+03,5.812000e+03
max,2.320947e+07,1.998000e+03,6.138000e+03,1.227636e+05,1.003420e+05,9.300000e+03,9.402000e+03


In [7]:
transaction_df.duplicated()

0           False
1           False
2           False
3           False
4           False
            ...  
12858749    False
12858750    False
12858751    False
12858752    False
12858753    False
Length: 12858754, dtype: bool

In [8]:
transaction_df.duplicated().sum()

0

In [4]:
transaction_df.isnull()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
12858749,False,False,False,False,False,False,False,False,True,True,False,True
12858750,False,False,False,False,False,False,False,False,False,False,False,True
12858751,False,False,False,False,False,False,False,False,True,True,False,True
12858752,False,False,False,False,False,False,False,False,False,False,False,True


In [9]:
transaction_df.isnull().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state     1509523
zip                1509523
mcc                      1
errors            12654526
dtype: int64

In [10]:
# creating mapping from city to state and city to zip where is not null
city_state_map = transaction_df.dropna(subset=['merchant_state']).set_index('merchant_city')['merchant_state'].to_dict()
city_zip_map = transaction_df.dropna(subset=['zip']).set_index('merchant_city')['zip'].to_dict()

# fill missing merchant state using mapping

transaction_df['merchant_state'] = transaction_df.apply(
    lambda row: city_state_map.get(row['merchant_city'], row['merchant_state']),
    axis=1
)


transaction_df['zip'] = transaction_df.apply(
    lambda row: city_zip_map.get(row['merchant_city'], row['zip']),
    axis=1
)

# handle mccc missing value
transaction_df['mcc'] = pd.to_numeric(transaction_df['mcc'], errors='coerce')
mcc_mean = transaction_df['mcc'].mean()
transaction_df['mcc'] = transaction_df['mcc'].fillna(mcc_mean)
print("\nAfter filling missing values:\n", transaction_df)


After filling missing values:
                 id                 date  client_id  card_id   amount  \
0          7475327  2022-02-19 05:51:55       1556     2972 -1386.00   
1          7475328  2023-01-13 02:58:58        561     4575   262.26   
2          7475329  2024-07-03 23:41:24       1129      102  1440.00   
3          7475331  2022-06-12 09:33:41        430     2860  3600.00   
4          7475332  2023-08-26 10:05:48        848     3915   835.38   
...            ...                  ...        ...      ...      ...   
12858749  23209463  2022-05-16 02:17:24        597     4540   126.00   
12858750  23209464  2023-07-23 01:05:29       1508     3279   593.46   
12858751  23209465  2023-05-03 10:17:19       1727     4918   284.04   
12858752  23209466  2022-05-15 15:12:30        467     4278   812.88   
12858753  23209467  2022-04-17 21:21:33       1384     3723  1014.48   

                    use_chip  merchant_id     merchant_city merchant_state  \
0          Swipe Transact

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [11]:
transaction_df.isnull().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state     1509523
zip                1509523
mcc                      0
errors            12654526
dtype: int64

## Card csv data

In [12]:
card_df.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Mastercard,Debit,5561765976957072,04/2025,623,YES,2,437310,09/2002,2008,No
1,2731,825,Mastercard,Debit,5681404028657264,08/2025,393,YES,2,395424,04/2014,2014,No
2,3701,825,Visa,Debit,4715119191099084,08/2025,719,YES,2,835452,07/2003,2004,No
3,42,825,Visa,Credit,4133211808921544,12/2026,693,NO,1,223200,01/2003,2012,No
4,4659,825,Mastercard,Prepaid,5469974597438641,10/2026,75,YES,1,504,09/2008,2009,No


In [13]:
card_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     6146 non-null   int64 
 1   client_id              6146 non-null   int64 
 2   card_brand             6146 non-null   object
 3   card_type              6146 non-null   object
 4   card_number            6146 non-null   int64 
 5   expires                6146 non-null   object
 6   cvv                    6146 non-null   int64 
 7   has_chip               6146 non-null   object
 8   num_cards_issued       6146 non-null   int64 
 9   credit_limit           6146 non-null   int64 
 10  acct_open_date         6146 non-null   object
 11  year_pin_last_changed  6146 non-null   int64 
 12  card_on_dark_web       6146 non-null   object
dtypes: int64(7), object(6)
memory usage: 624.3+ KB


In [14]:
card_df.describe()

,id,client_id,card_number,cvv,num_cards_issued,credit_limit,year_pin_last_changed
count,6146.000000,6146.000000,6.146000e+03,6146.000000,6146.000000,6.146000e+03,6146.000000
mean,3072.500000,994.939636,4.710118e+15,506.220794,1.503091,2.582549e+05,2013.436707
std,1774.341709,578.614626,1.383261e+15,289.431123,0.519191,2.162603e+05,4.270699
min,0.000000,0.000000,3.002159e+14,0.000000,1.000000,0.000000e+00,2002.000000
25%,1536.250000,492.250000,4.401621e+15,257.000000,1.000000,1.267695e+05,2010.000000
50%,3072.500000,992.000000,4.937990e+15,516.500000,1.000000,2.266650e+05,2013.000000
75%,4608.750000,1495.000000,5.514460e+15,756.000000,2.000000,3.448170e+05,2017.000000
max,6145.000000,1999.000000,6.986868e+15,999.000000,3.000000,2.722014e+06,2020.000000


In [7]:
card_df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
6141    False
6142    False
6143    False
6144    False
6145    False
Length: 6146, dtype: bool

In [15]:
card_df.duplicated().sum()

0

In [8]:
card_df.isnull()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,False,False,False,False,False,False,False,False,False,False,False,False,False
6142,False,False,False,False,False,False,False,False,False,False,False,False,False
6143,False,False,False,False,False,False,False,False,False,False,False,False,False
6144,False,False,False,False,False,False,False,False,False,False,False,False,False


In [9]:
card_df.isnull().sum()

id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
dtype: int64

## User csv data

In [15]:
users_df.head()

,id,current_age,retirement_age,birth_year,birth_month,gender,address,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,"126 Main Road, Durban",527004,1074528,2297034,787,5
1,1746,53,68,1966,12,Female,"254 Van der Merwe Avenue, Cape Town",682038,1390572,3444282,701,5
2,1718,81,67,1938,11,Female,"856 King George Street, Pietermaritzburg",408258,602694,3528,698,5
3,708,63,63,1957,1,Female,"565 Pretoria Street, Durban",2936610,4498650,3641904,722,4
4,1164,43,70,1976,9,Male,"269 Swart Street, Pretoria",968346,1974366,3309390,675,1


In [16]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 2000 non-null   int64 
 1   current_age        2000 non-null   int64 
 2   retirement_age     2000 non-null   int64 
 3   birth_year         2000 non-null   int64 
 4   birth_month        2000 non-null   int64 
 5   gender             2000 non-null   object
 6   address            2000 non-null   object
 7   per_capita_income  2000 non-null   int64 
 8   yearly_income      2000 non-null   int64 
 9   total_debt         2000 non-null   int64 
 10  credit_score       2000 non-null   int64 
 11  num_credit_cards   2000 non-null   int64 
dtypes: int64(10), object(2)
memory usage: 187.6+ KB


In [17]:
users_df.describe()

,id,current_age,retirement_age,birth_year,birth_month,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2.000000e+03,2.000000e+03,2.000000e+03,2000.000000,2000.000000
mean,999.500000,45.391500,66.237500,1973.803000,6.439000,4.165547e+05,8.228859e+05,1.146774e+06,709.734500,3.073000
std,577.494589,18.414092,3.628867,18.421234,3.565338,2.038345e+05,4.138671e+05,9.405802e+05,67.221949,1.637379
min,0.000000,18.000000,50.000000,1918.000000,1.000000,0.000000e+00,1.800000e+01,0.000000e+00,480.000000,1.000000
25%,499.750000,30.000000,65.000000,1961.000000,3.000000,3.028410e+05,5.907330e+05,4.317615e+05,681.000000,2.000000
50%,999.500000,44.000000,66.000000,1975.000000,7.000000,3.704580e+05,7.334010e+05,1.048518e+06,711.500000,3.000000
75%,1499.250000,58.000000,68.000000,1989.000000,10.000000,4.731480e+05,9.485730e+05,1.603269e+06,753.000000,4.000000
max,1999.000000,101.000000,79.000000,2002.000000,12.000000,2.936610e+06,5.526324e+06,9.292734e+06,850.000000,9.000000


In [11]:
users_df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Length: 2000, dtype: bool

In [14]:
users_df.duplicated().sum()

0

In [12]:
users_df.isnull()

,id,current_age,retirement_age,birth_year,birth_month,gender,address,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,False,False,False,False,False,False,False,False,False,False,False,False
1996,False,False,False,False,False,False,False,False,False,False,False,False
1997,False,False,False,False,False,False,False,False,False,False,False,False
1998,False,False,False,False,False,False,False,False,False,False,False,False


In [13]:
users_df.isnull().sum()

id                   0
current_age          0
retirement_age       0
birth_year           0
birth_month          0
gender               0
address              0
per_capita_income    0
yearly_income        0
total_debt           0
credit_score         0
num_credit_cards     0
dtype: int64

### JSON file

In [4]:
train_fraud_labels.head()

,transaction_id,is_fraud
0,10649266,No
1,23410063,No
2,9316588,No
3,12478022,No
4,9558530,No


In [6]:
train_fraud_labels.isnull().sum()

transaction_id    0
is_fraud          0
dtype: int64

In [7]:
# Save to Kaggle working directory
output_path = "/kaggle/working/train_fraud_labels.csv"
train_fraud_labels.to_csv(output_path, index=False)

print(f"File saved successfully at: {output_path}")

File saved successfully at: /kaggle/working/train_fraud_labels.csv
